<!--TUTORIAL INFORMATION-->
<img align="left" style="padding-right:10px;" src="images/mindboggle_logo_100px.jpg">
*This notebook provides a tutorial to the [Mindboggle software](http://mindboggle.info) and assumes that the Mindboggle docker container has been installed as per the [installation instructions](http://mindboggle.readthedocs.io/en/latest/#installing-mindboggle). Rather than show examples of how to run Mindboggle on the command line, here we will dig into the Python library to run individual functions (by [Arno Klein](http://binarybottle.com), [CC-BY](https://creativecommons.org/licenses/by/3.0/us/legalcode)).*

# Mindboggle tutorial

Mindboggle’s open source brain morphometry platform takes in preprocessed T1-weighted MRI data, and outputs volume, surface, and tabular data containing label, feature, and shape information for further analysis. In this tutorial, we will:

  - Look at Mindboggle's processing steps and output.
  - Run "mindboggle --help" on the command line in a Docker container.
  - Run "mindboggle" on the command line with example data.
  - Run individual functions within the mindboggle Python library.


## Mindboggle processing steps and output

Open Mindboggle's website summarizing the software's processing steps and output file tree.

In [ ]:
import webbrowser
url = 'http://mindboggle.readthedocs.io/en/latest/#mindboggle-processing-steps'
webbrowser.open(url)

## "mindboggle --help" on the command line

First, let's see what command-line options Mindboggle provides:

In [ ]:
%%bash

docker run --rm -i -v /:/root/data --entrypoint /bin/bash bids/mindboggle;

mindboggle --help

## "mindboggle" command with example data

In the following example, we will run Mindboggle on FreeSurfer (but not ANTs) data to get shape measures such as curvature, depth, and area for cortical surface labels, and to make it quick, we will turn off volume labels and shape measures, and surface-based Zernike moments and Laplace-Beltrami spectra.

In [ ]:
%%bash
docker run --rm -i -v /:/root/data --entrypoint /bin/bash bids/mindboggle;
HOST=/root/data/Users/arno;
FREESURFER_SUBJECT=$HOST/mindboggle_input_example/freesurfer/subjects/arno;
ANTS_SUBJECT=$HOST/mindboggle_input_example/ants/subjects/arno;
MINDBOGGLING=$HOST/mindboggling;
MINDBOGGLED=$HOST/mindboggled;

mindboggle $FREESURFER_SUBJECT --working $MINDBOGGLING --out $MINDBOGGLED \
           --no_volumes --no_sulci --no_points --no_moments --no_spectra

## Mindboggle library

Rather than call Mindboggle from the command line, we can also call individual Python functions within the Mindboggle library, which includes the following files:

  - mindboggle/mindboggle/
    - mio/  *-- input/output functions*
        - colors.py  *-- colormap-related functions*
        - convert_volumes.py  *-- read/write nifti volume files*
        - fetch_data.py  *-- fetch data from a URL or from third party software*
        - labels.py  *-- information about labeling protocols*
        - plots.py  *-- plot functions*
        - tables.py  *-- read/write tables*
        - vtks.py  *-- read/write VTK surface files*
    - guts/  *-- the "guts" underlying feature extraction and labeling code*
        - compute.py  *-- compute distances, etc.*
        - graph.py  *-- graph operations*
        - kernels.py  *-- kernels for graph operations*
        - mesh.py  *-- operate on surface mesh vertices*
        - paths.py  *-- connect surface mesh vertices*
        - rebound.py  *-- adjust label borders on a surface mesh*
        - relabel.py  *-- relabel surface or volume files*
        - segment.py  *-- segment a surface mesh*
    - features/  *-- higher-level feature extraction (folds, fundi, sulci)*
        - folds.py  *-- extract surface folds*
        - fundi.py  *-- extract fundus curves from folds*
        - sulci.py  *-- extract sulci from folds*
    - shapes/  *-- shape measurement functions
        - surface_shapes.py  *-- compute surface shapes (calls C++ library below)*
        - volume_shapes.py  *-- compute volumes and thicknesses*
        - laplace_beltrami.py  *-- compute a Laplace-Beltrami spectrum*
        - zernike/zernike.py  *-- compute Zernike moments of a collection of vertices*
        - likelihood.py  *-- compute (fundus) likelihood values*
    - thirdparty/  *-- third-party code*
        - ants.py  *-- call ANTs commands*
        - vtkviewer.py  *-- VTK viewer (by Hal Canary)*

  - mindboggle/vtk_cpp_tools  *-- C++ tools for measuring shapes on VTK surface files*
    - area/
    - curvature/
    - geodesic_depth/
    - gradient/
    - surface_overlap/
    - travel_depth/

## mindboggle Python functions

Let's run some functions within Mindboggle's Python library.  The following examples are excerpts from the above files' docstrings.

In [ ]:
# Compute histogram values from nibabel-readable image:
!docker run --rm -i -v /:/root/data --entrypoint /bin/bash bids/mindboggle;
from mindboggle.guts.compute import compute_image_histogram
from mindboggle.mio.fetch_data import prep_tests
urls, fetch_data = prep_tests()
labels_file = fetch_data(urls['freesurfer_labels'], '', '.nii.gz')
nbins = 100
threshold = 0.5
histogram_values = compute_image_histogram(labels_file, nbins, threshold)
histogram_values[0:5]


In [ ]:
# Compute the number of times each label occurs.
"""
    Parameters
    ----------
    labels : numpy 1-D array of integers
        labels (e.g., one label per vertex of a mesh)
    include_labels : list of integers
        labels to include
        (if empty, use unique numbers in image volume file)
    exclude_labels : list of integers
        labels to be excluded

    Returns
    -------
    unique_labels : list of integers
        unique label numbers
    counts : list of floats
        number of times each label occurs
"""
!docker run --rm -i -v /:/root/data --entrypoint /bin/bash bids/mindboggle;
from mindboggle.guts.compute import count_per_label
labels = [8,8,8,8,8,10,11,12,10,10,11,11,11,12,12,12,12,13]
include_labels = [9,10,11,12]
exclude_labels = [13]
unique_labels, counts = count_per_label(labels, include_labels, exclude_labels)
unique_labels
    [9, 10, 11, 12]
counts
    [0, 3, 4, 5]
import nibabel as nb
from mindboggle.mio.vtks import read_scalars
from mindboggle.mio.labels import DKTprotocol
from mindboggle.guts.compute import count_per_label
from mindboggle.mio.fetch_data import prep_tests
urls, fetch_data = prep_tests()
labels_file = fetch_data(urls['freesurfer_labels'], '', '.nii.gz')
img = nb.load(labels_file)
hdr = img.get_header()
labels = img.get_data().ravel()
dkt = DKTprotocol()
include_labels = dkt.label_numbers
exclude_labels = []
unique_labels, counts = count_per_label(labels,
    ...     include_labels, exclude_labels)
counts[0:10]
